In [ ]:
#old
from germansentiment import SentimentModel
model = SentimentModel()
texts = []
for index, row in df_raw.iloc[:20].iterrows():
    texts.append(df_raw.iloc[index]['Text'])
texts
result = model.predict_sentiment(texts)
print(result)